In [1]:
import json
from pathlib import Path
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from transformers import AutoTokenizer, TFAutoModelForQuestionAnswering

In [2]:
def read_squad(path):
    path = Path(path)
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    contexts = []
    questions = []
    answers = []
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                for answer in qa['answers']:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)

    return contexts, questions, answers

train_contexts, train_questions, train_answers = read_squad('input/squad/train-v2.0.json')
val_contexts, val_questions, val_answers = read_squad('input/squad/dev-v2.0.json')

In [3]:
def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)

        # sometimes squad answers are off by a character or two – fix this
        if context[start_idx:end_idx] == gold_text:
            answer['answer_end'] = end_idx
        elif context[start_idx-1:end_idx-1] == gold_text:
            answer['answer_start'] = start_idx - 1
            answer['answer_end'] = end_idx - 1     # When the gold label is off by one character
        elif context[start_idx-2:end_idx-2] == gold_text:
            answer['answer_start'] = start_idx - 2
            answer['answer_end'] = end_idx - 2     # When the gold label is off by two characters

add_end_idx(train_answers, train_contexts)
add_end_idx(val_answers, val_contexts)

In [4]:
pretrained_dir = "pretrained/distilbert-base-cased-distilled-squad"
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained(pretrained_dir, model_max_length=512)
print(repr(tokenizer))

PreTrainedTokenizerFast(name_or_path='pretrained/distilbert-base-cased-distilled-squad', vocab_size=28996, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


In [5]:
%%time
train_encodings = tokenizer(train_contexts, train_questions, max_length=512, truncation=True, padding=True)

CPU times: user 34 s, sys: 14.8 s, total: 48.8 s
Wall time: 16.8 s


In [6]:
%%time
val_encodings = tokenizer(val_contexts, val_questions, max_length=512, truncation=True, padding=True)

CPU times: user 9.12 s, sys: 3.44 s, total: 12.6 s
Wall time: 4.43 s


In [7]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))

        # if start position is None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length

    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, train_answers)
add_token_positions(val_encodings, val_answers)

In [8]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    {key: train_encodings[key] for key in ['input_ids', 'attention_mask']},
    {key: train_encodings[key] for key in ['start_positions', 'end_positions']}
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    {key: val_encodings[key] for key in ['input_ids', 'attention_mask']},
    {key: val_encodings[key] for key in ['start_positions', 'end_positions']}
))

In [9]:
# Keras will expect a tuple when dealing with labels
train_dataset = train_dataset.map(lambda x, y: (x, (y['start_positions'], y['end_positions'])))

In [10]:
%%time
from transformers import TFDistilBertForQuestionAnswering
model = TFDistilBertForQuestionAnswering.from_pretrained(pretrained_dir, return_dict=False)
print(repr(model.config))

All model checkpoint layers were used when initializing TFDistilBertForQuestionAnswering.

All the layers of TFDistilBertForQuestionAnswering were initialized from the model checkpoint at pretrained/distilbert-base-cased-distilled-squad.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForQuestionAnswering for predictions without further training.


DistilBertConfig {
  "_name_or_path": "pretrained/distilbert-base-cased-distilled-squad",
  "activation": "gelu",
  "architectures": [
    "DistilBertForQuestionAnswering"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "return_dict": false,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": true,
  "tie_weights_": true,
  "transformers_version": "4.8.1",
  "vocab_size": 28996
}

CPU times: user 1.77 s, sys: 1.8 s, total: 3.56 s
Wall time: 1.81 s


In [11]:
# Keras will assign a separate loss for each output and add them together. So we'll just use the standard CE loss
# instead of using the built-in model.compute_loss, which expects a dict of outputs and averages the two terms.
# Note that this means the loss will be 2x of when using TFTrainer since we're adding instead of averaging them.
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# if using 🤗 Transformers >3.02, make sure outputs are tuples
model.distilbert.return_dict = False


optimizer = keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=loss) # can also use any keras loss fn
model.summary()

Model: "tf_distil_bert_for_question_answering"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
distilbert (TFDistilBertMain multiple                  65190912  
_________________________________________________________________
qa_outputs (Dense)           multiple                  1538      
_________________________________________________________________
dropout_19 (Dropout)         multiple                  0         
Total params: 65,192,450
Trainable params: 65,192,450
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.fit(train_dataset.shuffle(1000).batch(16), epochs=2)

Epoch 1/2
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


TypeError: in user code:

    /mnt/s/dev/seahrh/kaggle-coleridge-initiative/venv/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:864 train_function  *
        return step_function(self, iterator)
    /mnt/s/dev/seahrh/kaggle-coleridge-initiative/venv/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:851 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /mnt/s/dev/seahrh/kaggle-coleridge-initiative/venv/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1308 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /mnt/s/dev/seahrh/kaggle-coleridge-initiative/venv/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2824 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /mnt/s/dev/seahrh/kaggle-coleridge-initiative/venv/lib/python3.7/site-packages/tensorflow/python/distribute/distribute